In [8]:
import configparser
import datetime
import os
import tkinter as tk
import mysql.connector
import pytz
from tkinter import messagebox, ttk

class SonoApp:
    """
    Aplicativo para registrar dados de sono em um banco de dados MySQL.
    """

    def __init__(self, root):
        """
        Inicializa a interface gráfica e configura a conexão com o banco de dados.

        Args:
            root (tk.Tk): A janela principal do aplicativo.
        """
        self.root = root
        self.root.title("Registro de Sono")

        # Configurações do MySQL
        self.config = configparser.ConfigParser()
        config_path = r"C:\Users\robso\PERFORMANCE_ESTUDOS2\Credenciais\config.ini"  # Caminho absoluto
        self.config.read(config_path)

        self.db_config = self._load_db_config()
        if not self.db_config:
            root.destroy()
            return

        # Fuso horário de Brasília
        self.brasilia_tz = pytz.timezone("America/Sao_Paulo")

        # Componentes da Interface Gráfica
        self.inicio_label = ttk.Label(root, text="Hora de Início (HH:MM) - Formato 24h:")
        self.inicio_hora_entry = ttk.Entry(root, width=5)
        self.inicio_minuto_entry = ttk.Entry(root, width=5)

        self.termino_label = ttk.Label(root, text="Hora de Término (HH:MM) - Formato 24h:")
        self.termino_hora_entry = ttk.Entry(root, width=5)
        self.termino_minuto_entry = ttk.Entry(root, width=5)

        self.acordou_label = ttk.Label(root, text="Vezes que Acordou:")
        self.acordou_entry = ttk.Entry(root, width=10)

        self.salvar_button = ttk.Button(
            root, text="Salvar Registro de Sono", command=self._salvar_sono
        )

        # Layout
        self.inicio_label.grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.inicio_hora_entry.grid(row=0, column=1, padx=5, pady=5, sticky="ew")
        self.inicio_minuto_entry.grid(row=0, column=2, padx=5, pady=5, sticky="ew")

        self.termino_label.grid(row=1, column=0, padx=5, pady=5, sticky="w")
        self.termino_hora_entry.grid(row=1, column=1, padx=5, pady=5, sticky="ew")
        self.termino_minuto_entry.grid(row=1, column=2, padx=5, pady=5, sticky="ew")

        self.acordou_label.grid(row=2, column=0, padx=5, pady=5, sticky="w")
        self.acordou_entry.grid(row=2, column=1, padx=5, pady=5, sticky="ew")

        self.salvar_button.grid(row=3, column=0, columnspan=3, padx=5, pady=5)

        root.columnconfigure(1, weight=1)
        root.columnconfigure(2, weight=1)

    def _load_db_config(self):
        """
        Carrega e valida a configuração do banco de dados do arquivo config.ini.

        Returns:
            dict: Um dicionário com as configurações do banco de dados.
                  Retorna None se houver um erro na configuração.
        """
        try:
            db_config = dict(self.config["database"])
        except KeyError:
            messagebox.showerror("Erro", "Seção 'database' não encontrada no config.ini")
            return None


        if "raise_on_warnings" in db_config:
            try:
                db_config["raise_on_warnings"] = self.config.getboolean(
                    "database", "raise_on_warnings"
                )
            except ValueError:
                messagebox.showerror(
                    "Erro",
                    "Valor inválido para raise_on_warnings no config.ini. "
                    "Use 'True' ou 'False'.",
                )
                return None
        return db_config

    def _validar_dados(self, inicio_hora_str, inicio_minuto_str,
                       termino_hora_str, termino_minuto_str, acordou_str):
        """
        Valida os dados inseridos pelo usuário.

        Args:
            inicio_hora_str (str): A hora de início do sono.
            inicio_minuto_str (str): O minuto de início do sono.
            termino_hora_str (str): A hora de término do sono.
            termino_minuto_str (str): O minuto de término do sono.
            acordou_str (str): O número de vezes que acordou.

        Returns:
            bool: True se os dados forem válidos, False caso contrário.
        """
        if not all([inicio_hora_str, inicio_minuto_str, termino_hora_str,
                    termino_minuto_str, acordou_str]):
            messagebox.showerror("Erro", "Por favor, preencha todos os campos.")
            return False

        try:
            inicio_hora = int(inicio_hora_str)
            inicio_minuto = int(inicio_minuto_str)
            termino_hora = int(termino_hora_str)
            termino_minuto = int(termino_minuto_str)
            vezes_acordou = int(acordou_str)  # noqa: F841 (usado abaixo)

            if not (0 <= inicio_hora <= 23 and 0 <= inicio_minuto <= 59
                    and 0 <= termino_hora <= 23 and 0 <= termino_minuto <= 59):
                raise ValueError("Hora ou minuto inválido.")

        except ValueError as e:
            messagebox.showerror("Erro", f"Valor inválido: {e}")
            return False

        return True

    def _criar_objetos_datetime(self, inicio_hora_str, inicio_minuto_str,
                                 termino_hora_str, termino_minuto_str):
        """
        Cria objetos datetime para início e término do sono.

        Args:
            inicio_hora_str (str): A hora de início do sono.
            inicio_minuto_str (str): O minuto de início do sono.
            termino_hora_str (str): A hora de término do sono.
            termino_minuto_str (str): O minuto de término do sono.

        Returns:
            tuple[datetime, datetime]: Uma tupla contendo os objetos
                                       datetime de início e término.
        """
        inicio_hora = int(inicio_hora_str)
        inicio_minuto = int(inicio_minuto_str)
        termino_hora = int(termino_hora_str)
        termino_minuto = int(termino_minuto_str)

        agora = datetime.datetime.now(self.brasilia_tz)
        hoje = agora.date()

        # Determina a data correta para o horário de início
        if 12 <= inicio_hora <= 23:
            # Se a hora de *início* está entre 12:00 e 23:59, usa a data de ontem
            data_inicio = hoje - datetime.timedelta(days=1)
        else:
            # Caso contrário, usa a data de hoje
            data_inicio = hoje

        inicio = datetime.datetime(
            data_inicio.year, data_inicio.month, data_inicio.day,
            inicio_hora, inicio_minuto
        )
        termino = datetime.datetime(
            hoje.year, hoje.month, hoje.day, termino_hora, termino_minuto
        )

        inicio = self.brasilia_tz.localize(inicio)
        termino = self.brasilia_tz.localize(termino)

        return inicio, termino

    def _salvar_sono_no_banco(self, inicio, termino, vezes_acordou):
        """
        Salva o registro de sono no banco de dados.

        Args:
            inicio (datetime): O objeto datetime de início do sono.
            termino (datetime): O objeto datetime de término do sono.
            vezes_acordou (int): O número de vezes que acordou durante o sono.

        Returns:
            bool: True se o registro for salvo com sucesso, False caso contrário.
        """
        try:
            with mysql.connector.connect(**self.db_config) as connection:
                with connection.cursor() as cursor:
                    query = """
                        INSERT INTO habitos.sono (inicio, termino, vezes_acordou)
                        VALUES (%s, %s, %s)
                    """
                    values = (
                        inicio.strftime("%Y-%m-%d %H:%M:%S"),
                        termino.strftime("%Y-%m-%d %H:%M:%S"),
                        vezes_acordou,
                    )
                    cursor.execute(query, values)
                connection.commit()
            messagebox.showinfo("Sucesso", "Registro de sono salvo com sucesso!")
            return True

        except mysql.connector.Error as err:
            messagebox.showerror("Erro", f"Erro ao salvar os dados: {err}")
            return False
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao salvar o registro de sono: {e}")
            return False

    def _salvar_sono(self):
        """
        Função principal para salvar o registro de sono.
        """
        inicio_hora_str = self.inicio_hora_entry.get()
        inicio_minuto_str = self.inicio_minuto_entry.get()
        termino_hora_str = self.termino_hora_entry.get()
        termino_minuto_str = self.termino_minuto_entry.get()
        acordou_str = self.acordou_entry.get()

        if not self._validar_dados(
            inicio_hora_str, inicio_minuto_str, termino_hora_str,
            termino_minuto_str, acordou_str
        ):
            return

        try:
            inicio, termino = self._criar_objetos_datetime(
                inicio_hora_str, inicio_minuto_str, termino_hora_str,
                termino_minuto_str
            )
        except ValueError as e:
            messagebox.showerror("Erro", f"Valor inválido: {e}")
            return

        try:
            vezes_acordou = int(acordou_str)
        except ValueError:
            messagebox.showerror("Erro", "Número de vezes que acordou inválido.")
            return

        if self._salvar_sono_no_banco(inicio, termino, vezes_acordou):
            # Limpa os campos após o salvamento
            self.inicio_hora_entry.delete(0, tk.END)
            self.inicio_minuto_entry.delete(0, tk.END)
            self.termino_hora_entry.delete(0, tk.END)
            self.termino_minuto_entry.delete(0, tk.END)
            self.acordou_entry.delete(0, tk.END)


if __name__ == "__main__":
    root = tk.Tk()
    style = ttk.Style(root)
    style.theme_use("clam")
    app = SonoApp(root)
    root.mainloop()